# LEZIONE 12 — Overfitting e Underfitting

## Obiettivi della Lezione

Al termine di questa lezione sarai in grado di:

1. Comprendere il trade-off tra bias e varianza.
2. Riconoscere i segnali di overfitting e underfitting nei dati.
3. Interpretare le curve di apprendimento (learning curves).
4. Diagnosticare problemi di generalizzazione nei modelli.
5. Scegliere la complessità appropriata per un modello.

## Prerequisiti

- Conoscenza di base di regressione e classificazione.
- Familiarità con train/test split e metriche semplici.
- Lettura di grafici di performance.

## Indice

1. Teoria concettuale approfondita
2. Schema mentale / mappa decisionale
3. Notebook dimostrativo
4. Metodi spiegati
5. Esercizi risolti (guidati)
6. Glossario
7. Errori comuni e debug rapido
8. Conclusione operativa
9. End-of-lesson checklist
10. Didactic changelog

## Librerie Utilizzate

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.model_selection import learning_curve, train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
```


# SEZIONE 1 — Teoria Concettuale Approfondita

## 1.1 — Il problema della generalizzazione

L'obiettivo del machine learning non è predire bene i dati già visti, ma predire bene dati nuovi. Questa capacità si chiama generalizzazione.

```
               TRAINING DATA                    NUOVI DATI
              ┌─────────────┐                 ┌─────────────┐
              │ ● ● ● ● ● ● │                 │ ? ? ? ? ? ? │
              │ ● ● ● ● ● ● │  ─► Modello ─►  │ ? ? ? ? ? ? │
              │ ● ● ● ● ● ● │                 │ ? ? ? ? ? ? │
              └─────────────┘                 └─────────────┘
                   Noti                          Ignoti

         Performance qui ≠ Performance qui!
```

## 1.2 — Due modi per fallire: underfitting e overfitting

### Underfitting (bias alto)

Il modello è troppo semplice e non cattura i pattern reali.

- Errore alto sul training set
- Errore alto sul test set
- I due errori sono simili

### Overfitting (varianza alta)

Il modello è troppo complesso e memorizza il rumore.

- Errore basso sul training set
- Errore alto sul test set
- Gap grande tra train e test


## 1.3 — Bias e varianza: il trade-off

L'errore totale di un modello può essere scomposto in tre componenti:

$$	ext{Errore Totale} = 	ext{Bias}^2 + 	ext{Varianza} + 	ext{Rumore Irriducibile}$$

**Spiegazione in parole semplici:**
- Bias misura quanto il modello è sistematicamente lontano dal vero.
- Varianza misura quanto il modello cambia se cambiano i dati di training.
- Il rumore irriducibile è la parte che non possiamo eliminare.

Collegheremo questa formula al codice quando confronteremo modelli con diverse complessità.

## 1.4 — Learning curves: lettura pratica

Le learning curves mostrano come cambia la performance al variare della quantità di dati di training. Confrontando train e validation possiamo diagnosticare:

- Underfitting: entrambi i punteggi sono bassi e vicini.
- Overfitting: train alto e validation più basso con gap ampio.
- Buona generalizzazione: entrambi alti e vicini.

## 1.5 — Segnali pratici e rimedi

**Segnali tipici:**
- Overfitting: accuracy training molto alta e validation molto più bassa.
- Underfitting: performance deludente anche sul training set.

**Rimedi principali:**
- Overfitting: ridurre complessità, regolarizzare, aumentare dati.
- Underfitting: aumentare complessità, aggiungere feature, migliorare preprocessing.


# SEZIONE 2 — Schema Mentale / Mappa Decisionale

## Diagnosi rapida: albero decisionale

```
HO ADDESTRATO UN MODELLO
          │
          ▼
    ┌─────────────────┐
    │ Train score     │
    │    alto?        │
    └────────┬────────┘
             │
     ┌───────┴───────┐
     │               │
    NO              SI
     │               │
     ▼               ▼
┌────────────┐   ┌─────────────┐
│UNDERFITTING│   │Test score    │
│            │   │alto?         │
│Aumenta     │   └─────┬────────┘
│complessita │         │
└────────────┘   ┌─────┴───────┐
                 │             │
                SI            NO
                 │             │
                 ▼             ▼
          ┌──────────┐   ┌─────────────┐
          │ OTTIMO   │   │ OVERFITTING │
          │          │   │             │
          │Generalizza│  │ Riduci      │
          └──────────┘   │complessita  │
                         │o piu dati   │
                         └─────────────┘
```

## Checklist operativa

- Ho abbastanza dati rispetto al numero di feature?
- Ho fatto lo split train/test prima del preprocessing?
- Il gap train-test è accettabile (circa 5-10%)?
- La cross-validation mostra variabilità contenuta?


# SEZIONE 3 — Notebook Dimostrativo

## 3.1 — Bias vs varianza con regressione polinomiale

**Perché lo facciamo:** vogliamo vedere come cambia la complessita del modello e come si riflette su bias e varianza. Questo collega la teoria al grafico finale.

**Cosa mostra il grafico:** tre curve di modello con gradi diversi, sovrapposte ai dati reali.

**Perché conta:** la forma della curva mostra se il modello e troppo semplice (underfitting) o troppo complesso (overfitting).

**Come interpretarlo:**
- Curva quasi lineare e lontana dai dati: underfitting.
- Curva molto ondulata che passa per ogni punto: overfitting.


In [ ]:
# Dimostrazione: Bias vs Varianza con Regressione Polinomiale

# Importiamo le librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

# Impostiamo il seed per riproducibilita dei risultati
np.random.seed(42)

# Definiamo la funzione vera (che il modello non conosce)
def true_function(x):
    # Funzione non lineare usata per generare i dati
    return np.sin(1.5 * x)

# Generiamo dati con rumore
n_samples = 30
X = np.sort(np.random.uniform(0, 5, n_samples))
y = true_function(X) + np.random.normal(0, 0.3, n_samples)

# Trasformiamo X in matrice 2D per scikit-learn
X = X.reshape(-1, 1)

# Micro-checkpoint: verifichiamo dimensioni e assenza di NaN
assert X.shape[0] == y.shape[0], "X e y devono avere lo stesso numero di righe"
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"NaN in X: {np.isnan(X).any()}, NaN in y: {np.isnan(y).any()}")

# Punti per la visualizzazione continua
X_plot = np.linspace(0, 5, 200).reshape(-1, 1)

# Confrontiamo 3 gradi di polinomio
degrees = [1, 4, 15]
titles = [
    "Grado 1: UNDERFITTING (Bias alto, Varianza bassa)",
    "Grado 4: Bilanciato (Bias e Varianza moderati)",
    "Grado 15: OVERFITTING (Bias basso, Varianza alta)",
]

# Creiamo i grafici
fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=True)

for ax, degree, title in zip(axes, degrees, titles):
    # Creiamo una pipeline: trasformazione polinomiale + regressione lineare
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

    # Alleniamo il modello
    model.fit(X, y)

    # Predizioni su dati continui
    y_plot = model.predict(X_plot)

    # Calcoliamo l'errore sui dati di training
    y_train_pred = model.predict(X)
    mse = mean_squared_error(y, y_train_pred)

    # Visualizziamo dati e curva del modello
    ax.scatter(X, y, color="black", label="Dati")
    ax.plot(X_plot, y_plot, color="blue", label="Modello")
    ax.set_title(f"{title}
MSE train: {mse:.3f}")
    ax.set_xlabel("X")
    ax.set_ylabel("y")

# Output atteso: 3 grafici che mostrano l'effetto della complessita
plt.tight_layout()
plt.show()


## 3.2 — Learning curves per diagnosticare overfitting

**Perché lo facciamo:** vogliamo vedere la distanza tra train e validation al crescere dei dati, cosi possiamo diagnosticare underfitting o overfitting.

**Cosa mostra il grafico:** per ogni modello, due linee che indicano accuracy su train e validation al crescere dei campioni.

**Perché conta:** il gap tra le due curve indica quanta varianza ha il modello.

**Come interpretarlo:**
- Linee basse e vicine: underfitting.
- Linee alte ma distanti: overfitting.


In [ ]:
# Dimostrazione: Learning Curves per Diagnosticare Overfitting

# Importiamo le librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Creiamo un dataset di classificazione
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    flip_y=0.1,
    random_state=42,
)

# Micro-checkpoint: controlliamo le dimensioni e la distribuzione delle classi
assert X.shape[0] == y.shape[0], "X e y devono avere lo stesso numero di righe"
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Distribuzione classi: {np.bincount(y)}")
print(f"NaN in X: {np.isnan(X).any()}, NaN in y: {np.isnan(y).any()}")

# Definiamo tre modelli con complessita crescente
models = {
    "Logistic Regression (Possibile underfitting)": LogisticRegression(max_iter=1000),
    "Decision Tree depth=5 (Bilanciato)": DecisionTreeClassifier(max_depth=5, random_state=42),
    "Decision Tree depth=None (Possibile overfitting)": DecisionTreeClassifier(max_depth=None, random_state=42),
}

# Calcoliamo e visualizziamo le learning curves
fig, axes = plt.subplots(1, 3, figsize=(18, 4), sharey=True)

for ax, (label, model) in zip(axes, models.items()):
    train_sizes, train_scores, val_scores = learning_curve(
        estimator=model,
        X=X,
        y=y,
        train_sizes=np.linspace(0.1, 1.0, 5),
        cv=5,
        scoring="accuracy",
        n_jobs=None,
    )

    # Calcoliamo media e deviazione standard
    train_mean = train_scores.mean(axis=1)
    val_mean = val_scores.mean(axis=1)

    # Visualizziamo le curve
    ax.plot(train_sizes, train_mean, "o-", label="Train score")
    ax.plot(train_sizes, val_mean, "o-", label="Validation score")
    ax.set_title(label)
    ax.set_xlabel("Dimensione del training")
    ax.set_ylabel("Accuracy")
    ax.legend()

# Output atteso: 3 pannelli con curve train/validation
plt.tight_layout()
plt.show()


## 3.3 — Train vs test score al variare della complessita

**Perché lo facciamo:** vogliamo osservare come un iperparametro di complessita (max_depth) cambia il gap tra train e test.

**Cosa mostra il grafico:** due linee (train e test) al variare della profondita dell'albero.

**Perché conta:** individuiamo la zona in cui il test smette di migliorare mentre il train continua a crescere.

**Come interpretarlo:**
- Curve vicine e alte: buona generalizzazione.
- Train alto e test che cala: overfitting.


In [ ]:
# Dimostrazione: Train vs Test Score al Variare della Complessita

# Importiamo le librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Creiamo il dataset
X, y = make_classification(
    n_samples=500,
    n_features=15,
    n_informative=8,
    flip_y=0.1,
    random_state=42,
)

# Split train/test (prima del training, per evitare leakage)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Micro-checkpoint: controlliamo dimensioni e distribuzione
assert X_train.shape[0] + X_test.shape[0] == X.shape[0], "Split incoerente"
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"Distribuzione classi train: {np.bincount(y_train)}")
print(f"Distribuzione classi test: {np.bincount(y_test)}")

# Testiamo diversi valori di max_depth
max_depth_values = range(1, 21)
train_scores = []
test_scores = []

for depth in max_depth_values:
    # Creiamo e addestriamo il modello
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model.fit(X_train, y_train)

    # Valutiamo su train e test
    train_scores.append(model.score(X_train, y_train))
    test_scores.append(model.score(X_test, y_test))

# Visualizziamo il gap train-test
plt.figure(figsize=(8, 4))
plt.plot(max_depth_values, train_scores, label="Train score")
plt.plot(max_depth_values, test_scores, label="Test score")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.title("Train vs Test score al variare della complessita")
plt.legend()
plt.show()


# SEZIONE 4 — Metodi Spiegati

Di seguito i metodi principali usati nella lezione. Per ciascuno indichiamo cosa fa, input/output, errori tipici e quando usarlo.

## `make_classification`
- Cosa fa: genera un dataset sintetico di classificazione.
- Input: parametri scalari (n_samples, n_features, random_state).
- Output: `X` ndarray (n_samples, n_features), `y` ndarray (n_samples,).
- Errori tipici: classi troppo sbilanciate se `weights` non impostato.
- Quando usarlo: demo e test rapidi; non usarlo per valutazioni reali.

## `train_test_split`
- Cosa fa: divide i dati in train e test.
- Input: `X` (n_samples, n_features), `y` (n_samples,).
- Output: `X_train`, `X_test`, `y_train`, `y_test`.
- Errori tipici: dimenticare `stratify=y` in classificazione sbilanciata.
- Quando usarlo: sempre per una prima validazione.

## `learning_curve`
- Cosa fa: calcola performance su train e validation al crescere dei dati.
- Input: estimator, `X`, `y`, `cv`.
- Output: `train_sizes`, `train_scores`, `val_scores`.
- Errori tipici: interpretare male il gap train/validation.
- Quando usarlo: diagnosi di overfitting/underfitting.

## `PolynomialFeatures` + `LinearRegression`
- Cosa fa: crea feature polinomiali e adatta una regressione lineare.
- Input: `X` 2D, `y` 1D.
- Output: modello che predice valori continui.
- Errori tipici: usare `X` 1D senza reshape.
- Quando usarlo: mostrare l'effetto della complessita in regressione.

## `DecisionTreeClassifier`
- Cosa fa: albero di decisione per classificazione.
- Input: `X` 2D, `y` 1D.
- Output: `model` addestrato con `predict` e `score`.
- Errori tipici: albero troppo profondo senza regolarizzazione.
- Quando usarlo: baseline interpretabile, non quando serve stabilita.

## `cross_val_score`
- Cosa fa: misura performance con cross-validation.
- Input: estimator, `X`, `y`, `cv`.
- Output: array di score per ciascun fold.
- Errori tipici: usare metriche non appropriate al task.
- Quando usarlo: per verificare stabilita del modello.


# SEZIONE 5 — Esercizi Risolti (Guidati)

Gli esercizi sono completamente risolti con guida passo-passo.

## Esercizio 12.1: Identificare Overfitting vs Underfitting

**Obiettivo:** dato un modello e le sue performance, diagnosticare se c'e overfitting, underfitting o buona generalizzazione.

**Procedura:**
1. Creare un dataset.
2. Allenare tre modelli con complessita diversa.
3. Calcolare train/test score e gap.
4. Diagnosticare ogni modello.


**Perché lo facciamo:** vogliamo confrontare modelli con complessita diversa usando gli stessi dati, per capire come leggere il gap train-test.


In [ ]:
# ============================================================================
# ESERCIZIO 12.1 — Identificare Overfitting vs Underfitting
# ============================================================================

# Importiamo le librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# STEP 1: Creiamo il dataset
print("STEP 1: Creazione Dataset")
print("-" * 50)

X, y = make_classification(
    n_samples=800,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    flip_y=0.05,
    random_state=42,
)

# Micro-checkpoint: controlliamo dimensioni e distribuzione classi
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Distribuzione classi: {np.bincount(y)}")

# STEP 2: Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# STEP 3: Definiamo tre modelli con complessita diversa
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree depth=3": DecisionTreeClassifier(max_depth=3, random_state=42),
    "Decision Tree depth=None": DecisionTreeClassifier(max_depth=None, random_state=42),
}

# STEP 4: Valutazione e diagnosi
print("
Confronto modelli")
print("-" * 50)

for name, model in models.items():
    # Addestriamo il modello
    model.fit(X_train, y_train)

    # Calcoliamo score su train e test
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    gap = train_score - test_score

    # Cross-validation per stabilita
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)

    print(f"{name}")
    print(f"  Train score: {train_score:.3f}")
    print(f"  Test score:  {test_score:.3f}")
    print(f"  Gap:         {gap:.3f}")
    print(f"  CV mean/std: {cv_scores.mean():.3f} / {cv_scores.std():.3f}
")


---

## Esercizio 12.2: Learning Curve Completa

**Obiettivo:** costruire e interpretare una learning curve per diagnosticare problemi di generalizzazione.

**Procedura:**
1. Creare un dataset.
2. Generare la learning curve con `learning_curve` di sklearn.
3. Visualizzare train e validation score.
4. Determinare se servono piu dati o un modello diverso.

**Cosa mostra il grafico:** l'andamento delle performance su train e validation al crescere dei dati.

**Come interpretarlo:**
- Se le curve sono lontane, serve ridurre la complessita.
- Se entrambe sono basse, serve un modello piu espressivo o nuove feature.


**Perché lo facciamo:** una learning curve rende visibile il compromesso tra bias e varianza e aiuta a decidere la prossima azione.


In [ ]:
# ============================================================================
# ESERCIZIO 12.2 — Learning Curve Completa
# ============================================================================

# Importiamo le librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier

# STEP 1: Creiamo il dataset
print("STEP 1: Creazione Dataset")
print("-" * 50)

X, y = make_classification(
    n_samples=2000,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    flip_y=0.08,
    random_state=42,
)

# Micro-checkpoint: confermiamo dimensioni e distribuzione
print(f"Dataset: {X.shape[0]} campioni, {X.shape[1]} feature")
print(f"Distribuzione classi: {np.bincount(y)}")

# STEP 2: Definiamo il modello
model = RandomForestClassifier(n_estimators=200, random_state=42)

# STEP 3: Calcoliamo la learning curve
train_sizes, train_scores, val_scores = learning_curve(
    estimator=model,
    X=X,
    y=y,
    train_sizes=np.linspace(0.1, 1.0, 6),
    cv=5,
    scoring="accuracy",
)

# STEP 4: Calcoliamo le medie
train_mean = train_scores.mean(axis=1)
val_mean = val_scores.mean(axis=1)

# STEP 5: Visualizziamo le curve
plt.figure(figsize=(8, 4))
plt.plot(train_sizes, train_mean, "o-", label="Train score")
plt.plot(train_sizes, val_mean, "o-", label="Validation score")
plt.xlabel("Dimensione del training")
plt.ylabel("Accuracy")
plt.title("Learning Curve - Random Forest")
plt.legend()
plt.show()


---

## Esercizio 12.3: Trovare la Complessita Ottimale

**Obiettivo:** usare la validation per trovare il valore ottimale di un iperparametro.

**Procedura:**
1. Testare diversi valori di `max_depth` per un Decision Tree.
2. Per ogni valore, calcolare train e test score.
3. Identificare il punto di overfitting.
4. Scegliere il valore ottimale.

**Cosa mostra il grafico:** le curve di train, test e cross-validation al variare di `max_depth`.

**Come interpretarlo:**
- Se il train continua a salire ma test e CV si fermano, siamo oltre il punto ottimale.
- Il valore ottimale e vicino al massimo della curva di test o CV.


**Perché lo facciamo:** osservare il punto in cui il train continua a salire ma il test si stabilizza o scende.


In [ ]:
# ============================================================================
# ESERCIZIO 12.3 — Trovare la Complessita Ottimale
# ============================================================================

# Importiamo le librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier

# STEP 1: Creiamo il dataset
print("STEP 1: Creazione Dataset")
print("-" * 50)

X, y = make_classification(
    n_samples=1500,
    n_features=15,
    n_informative=8,
    n_redundant=4,
    flip_y=0.08,
    random_state=42,
)

# STEP 2: Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Micro-checkpoint: controlliamo dimensioni
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")

# STEP 3: Testiamo diversi valori di max_depth
max_depth_values = range(1, 21)
train_scores = []
test_scores = []
cv_means = []

for depth in max_depth_values:
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model.fit(X_train, y_train)

    train_scores.append(model.score(X_train, y_train))
    test_scores.append(model.score(X_test, y_test))

    cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    cv_means.append(cv_scores.mean())

# STEP 4: Visualizziamo i risultati
plt.figure(figsize=(8, 4))
plt.plot(max_depth_values, train_scores, label="Train score")
plt.plot(max_depth_values, test_scores, label="Test score")
plt.plot(max_depth_values, cv_means, label="CV mean")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.title("Scelta della complessita ottimale")
plt.legend()
plt.show()


# SEZIONE 6 — Glossario

- **Generalizzazione**: capacita di un modello di funzionare bene su dati nuovi.
- **Overfitting**: modello troppo complesso che memorizza il rumore.
- **Underfitting**: modello troppo semplice che non cattura i pattern.
- **Bias**: errore sistematico dovuto a un modello troppo semplice.
- **Varianza**: sensibilita del modello a variazioni nei dati di training.
- **Rumore irriducibile**: componente dell'errore non eliminabile.
- **Learning curve**: grafico che mostra performance al crescere dei dati.
- **Train score**: performance del modello sui dati di training.
- **Validation score**: performance stimata su dati non visti (cross-validation).
- **Gap train-test**: differenza tra performance su train e test.
- **Iperparametro**: parametro scelto prima dell'addestramento (es. max_depth).
- **Cross-validation**: valutazione ripetuta su piu split.
- **Regolarizzazione**: tecniche che limitano la complessita del modello.
- **Dataset sintetico**: dati generati artificialmente per esercizi.
- **Decision tree**: modello a regole che cresce con la profondita.


# SEZIONE 7 — Errori Comuni e Debug Rapido

| Sintomo | Causa probabile | Quick fix |
|---|---|---|
| Train molto alto, test molto basso | Modello troppo complesso | Riduci `max_depth`, aumenta dati, regolarizza |
| Train e test entrambi bassi | Modello troppo semplice | Aumenta complessita o feature |
| Learning curve piatta e bassa | Bias alto | Cambia modello o aggiungi feature |
| Learning curve con grande gap | Varianza alta | Più dati o modello meno complesso |
| Risultati non riproducibili | Seed mancante | Imposta `random_state` o `np.random.seed` |


# SEZIONE 8 — Conclusione Operativa

## Concetti chiave

- Bias e varianza spiegano il trade-off tra semplicita e complessita del modello.
- Overfitting si riconosce da un grande gap tra train e validation/test.
- Le learning curves aiutano a decidere se servono piu dati o un modello diverso.

## Cosa applicare subito

1. Misura sempre train e test score insieme.
2. Usa la cross-validation per verificare stabilita.
3. Se il gap e grande, riduci complessita o aumenta dati.


# SEZIONE 9 — End-of-lesson Checklist

- So distinguere underfitting e overfitting con train/test score.
- So leggere una learning curve e interpretarene il gap.
- So collegare bias e varianza alla complessita del modello.
- So proporre un rimedio pratico in base al problema.


# SEZIONE 10 — Didactic Changelog

- Standardizzata la struttura con sezioni numerate e titolo coerente.
- Aggiunti prerequisiti, indice e librerie utilizzate.
- Inserita mappa decisionale per la diagnosi rapida.
- Aggiunte motivazioni prima di ogni blocco di codice.
- Inseriti micro-checkpoint con controlli su shape e classi.
- Rafforzate le spiegazioni su bias, varianza e learning curve.
- Aggiunta sezione Metodi Spiegati con input/output e errori tipici.
- Aggiunti glossario e tabella errori comuni.
- Inserita checklist finale operativa.
